#  Машинный перевод с использованием рекуррентных нейронных сетей

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Deep Learning with PyTorch (2020) Авторы: Eli Stevens, Luca Antiga, Thomas Viehmann
* https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html#the-seq2seq-model
* https://www.adeveloperdiary.com/data-science/deep-learning/nlp/machine-translation-recurrent-neural-network-pytorch/

## Задачи для совместного разбора

1\. Рассмотрите пример архитектуры Encoder-Decoder с использованием RNN. Обсудите концепцию teacher forcing.

In [794]:
import torch as th
import torch.nn as nn

In [795]:
n_ru_tokens = 1000
batch_size = 16
ru_seq_len = 15
n_en_tokens = 500
en_seq_len = 10

ru = th.randint(0, n_ru_tokens, size=(batch_size, ru_seq_len))
en = th.randint(0, n_en_tokens, size=(batch_size, en_seq_len))

In [796]:
class Encoder(nn.Module):
  def __init__(self, embedding_dim, hidden_size):
      super().__init__()
      self.emb = nn.Embedding(
          num_embeddings=n_ru_tokens,
          embedding_dim=embedding_dim,
          padding_idx=0
      )
      self.dropout = nn.Dropout(p=0.5)
      self.rnn = nn.GRU(embedding_dim, hidden_size, batch_first=True)

  def forward(self, X):
    out = self.emb(X) # batch x seq x emb_size
    out = self.dropout(out)
    _, h = self.rnn(out) # out: batch x seq x hidden_size
    return h # 1 x batch x hidden_size

In [797]:
embedding_dim = 100
encoder_hidden_size = 300

encoder = Encoder(embedding_dim, encoder_hidden_size)

encoder_output = encoder(ru)

In [798]:
encoder_output.shape

torch.Size([1, 16, 300])

In [799]:
class Decoder(nn.Module):
  def __init__(self, embedding_dim, decoder_hidden_size):
    super().__init__()
    self.emb = nn.Embedding(
          num_embeddings=n_en_tokens,
          embedding_dim=embedding_dim,
          padding_idx=0
    )
    self.rnn = nn.GRUCell(embedding_dim, decoder_hidden_size)
    self.fc = nn.Linear(decoder_hidden_size, n_en_tokens)

  def forward(self, encoder_output, labels):
    # labels: batch x seq_len - считаем, что в 0 столбце SOS
    # encoder_output: 1 x batch x encoder_hidden_size
    seq_len = labels.size(1)
    input_tokens = labels[:, 0]
    decoder_hidden = encoder_output[0]
    for _ in range(1, seq_len):
      out = self.emb(input_tokens).relu() # batch x emb_size
      decoder_hidden = self.rnn(out, decoder_hidden) # batch x dec_hidden
      out = self.fc(decoder_hidden) # batch x n_en_tokens

      # teacher forcing
      input_tokens = out.argmax(dim=1).detach()
      # ...


    # вернуть прогнозы для каждого эл-та последовательности
    # batch x seq x n_en_token
    return ...

In [800]:
decoder = Decoder(embedding_dim=100, decoder_hidden_size=300)
decoder(encoder_output, en)

Ellipsis

## Задачи для самостоятельного решения

<p class="task" id="1"></p>

1\. Считайте файлы `RuBQ_2.0_train.json` (обучающее множество) и `RuBQ_2.0_test.json` (тестовое множество). Для каждого файла создайте по списка: список предложений на русском языке и список предложений на английском языке. Выведите на экран количество примеров в обучающей и тестовой выборке.

- [x] Проверено на семинаре

In [801]:
import json
with open('/content/drive/MyDrive/пм21_финашка/3 курс/NLP/04_rnn/RuBQ_2.0_test.json') as json_file:
    data_test = json.load(json_file)

with open('/content/drive/MyDrive/пм21_финашка/3 курс/NLP/04_rnn/RuBQ_2.0_train.json') as json_file:
    data_train = json.load(json_file)

data_train[:5]

[{'uid': 0,
  'question_text': 'Что может вызвать цунами?',
  'query': 'SELECT ?answer \nWHERE {\n  wd:Q8070 wdt:P828 ?answer\n}',
  'answer_text': 'Землетрясение',
  'question_uris': ['http://www.wikidata.org/entity/Q8070'],
  'question_props': ['wdt:P828'],
  'answers': [{'type': 'uri',
    'label': 'землетрясение',
    'value': 'http://www.wikidata.org/entity/Q7944',
    'wd_names': {'ru': ['землетрясение', 'җир тетрәве'],
     'en': ['seism',
      'earthquake',
      'seismic activity',
      'fore shocks',
      'tremor',
      'earthquakes',
      'earth quake',
      'earthtemblor',
      'foreshock',
      'aftershock',
      'quake',
      'temblor',
      'earth temblor',
      'foreshocks',
      'after shocks',
      'earth quakes',
      'after shock',
      'earthtremor',
      'convulsion',
      'earth tremor',
      'shock',
      'fore shock',
      'aftershocks']},
    'wp_names': ['землетрясениям']},
   {'type': 'uri',
    'label': 'метеорит',
    'value': 'http://

In [802]:
train_ru = []
train_en = []
for i in data_train:
  train_ru.append(i['question_text'])
  train_en.append(i['question_eng'])

test_ru = []
test_en = []
for i in data_test:
  test_ru.append(i['question_text'])
  test_en.append(i['question_eng'])

In [803]:
train_ru[:10], train_en[:10]

(['Что может вызвать цунами?',
  'Кто написал роман «Хижина дяди Тома»?',
  'Кто автор пьесы «Ромео и Джульетта»?',
  'Как называется столица Румынии?',
  'На каком инструменте играл Джимми Хендрикс?',
  'Какой стране принадлежит остров Таити?',
  'Кто создал Зимний дворец в Санкт-Петербурге?',
  'Кто режиссер фильма "Бриллиантовая рука"?',
  'Какой стране принадлежат Канарские острова?',
  'Какой географический объект называется Лимпопо?'],
 ['What can cause a tsunami?',
  'Who wrote the novel "uncle Tom\'s Cabin"?',
  'Who is the author of the play "Romeo and Juliet"?',
  'What is the name of the capital of Romania?',
  'What instrument did Jimi Hendrix play?',
  'What country owns the island of Tahiti?',
  'Who created the Winter Palace in St. Petersburg?',
  'Who is the Director of the film "diamond hand"?',
  'Which country does the Canary Islands belong to?',
  'What geographical feature is called Limpopo?'])

In [804]:
len(train_ru), len(test_ru)

(2330, 580)

In [805]:
len(train_ru), len(train_en)

(2330, 2330)

<p class="task" id="2"></p>

2\. Создайте два Vocab на основе загруженных данных: `ru_vocab` для слов на русском языке и `en_vocab` для слов на английском языке (словари создаются на основе обучающего множества). Добавьте в словари специальные токены `<PAD>`, `<SOS>`, `<EOS>`. Выведите на экран количество токенов в полученных словарях. Выведите на экран максимальное кол-во слов в предложениях на русском языке и в предложениях на английском языке (в обучающей выборке).

- [x] Проверено на семинаре

In [806]:
from torchtext.vocab import Vocab
from torchtext.vocab import build_vocab_from_iterator

import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [807]:
import re
pattern_ru = re.compile(r'[^А-Яа-яёЁ]+')
pattern_en = re.compile(r'[^A-Za-z]+')

In [808]:
def del_punctuation(df, pattern):
  res = []
  for i in df:
    stroka = re.sub(pattern,' ', i)
    res.append(word_tokenize(stroka))

  return res

In [809]:
corpus_ru = del_punctuation(train_ru, pattern_ru)
corpus_en = del_punctuation(train_en, pattern_en)

corpus_ru_test = del_punctuation(test_ru, pattern_ru)
corpus_en_test = del_punctuation(test_en, pattern_en)

In [810]:
corpus_ru[:10], corpus_en[:10]

([['Что', 'может', 'вызвать', 'цунами'],
  ['Кто', 'написал', 'роман', 'Хижина', 'дяди', 'Тома'],
  ['Кто', 'автор', 'пьесы', 'Ромео', 'и', 'Джульетта'],
  ['Как', 'называется', 'столица', 'Румынии'],
  ['На', 'каком', 'инструменте', 'играл', 'Джимми', 'Хендрикс'],
  ['Какой', 'стране', 'принадлежит', 'остров', 'Таити'],
  ['Кто', 'создал', 'Зимний', 'дворец', 'в', 'Санкт', 'Петербурге'],
  ['Кто', 'режиссер', 'фильма', 'Бриллиантовая', 'рука'],
  ['Какой', 'стране', 'принадлежат', 'Канарские', 'острова'],
  ['Какой', 'географический', 'объект', 'называется', 'Лимпопо']],
 [['What', 'can', 'cause', 'a', 'tsunami'],
  ['Who', 'wrote', 'the', 'novel', 'uncle', 'Tom', 's', 'Cabin'],
  ['Who',
   'is',
   'the',
   'author',
   'of',
   'the',
   'play',
   'Romeo',
   'and',
   'Juliet'],
  ['What', 'is', 'the', 'name', 'of', 'the', 'capital', 'of', 'Romania'],
  ['What', 'instrument', 'did', 'Jimi', 'Hendrix', 'play'],
  ['What', 'country', 'owns', 'the', 'island', 'of', 'Tahiti'],
  ['W

In [811]:
len(corpus_ru), len(corpus_en)

(2330, 2330)

In [812]:
token = ['<PAD>', '<SOS>', '<EOS>']

vocab_ru = build_vocab_from_iterator(
    corpus_ru,
    specials = token
)

vocab_en = build_vocab_from_iterator(
    corpus_en,
    specials = token
)

len(vocab_ru), len(vocab_en)

(5959, 4348)

In [813]:
token = ['<PAD>', '<SOS>', '<EOS>']

vocab_ru_test = build_vocab_from_iterator(
    corpus_ru_test,
    specials = token
)

vocab_en_test = build_vocab_from_iterator(
    corpus_en_test,
    specials = token
)

len(vocab_ru_test), len(vocab_en_test)

(1973, 1598)

In [814]:
# vocab_ru.get_stoi()

In [815]:
max([len(i) for i in corpus_ru]), max([len(i) for i in corpus_en])

(25, 31)

In [816]:
maxlen = max(max([len(i) for i in corpus_ru]), max([len(i) for i in corpus_en])) +2
maxlen

33

<p class="task" id="3"></p>

3\. Создайте класс `RuEnDataset`. Реализуйте `__getitem__` таким образом, чтобы он возвращал кортеж `(x, y)`, где x - это набор индексов токенов для предложений на русском языке, а `y` - набор индексов токенов для предложений на английском языке. Используя преобразования, сделайте длины наборов индексов одинаковой фиксированной длины, добавьте в начало каждого набора индекс `<SOS>`, а в конец - индекс токена `<EOS>`. Создайте датасет для обучающей и тестовой выборки.

- [ ] Проверено на семинаре *(по-другому сделала)*

In [817]:
import torchtext.transforms as T

class RuEnDataset:

  def __init__(self, x, y, vocab_ru, vocab_en, maxlen):
    self.x = x  # corpus_ru
    self.y = y  # corpus_en
    self.vocab_ru = vocab_ru
    self.vocab_en = vocab_en
    self.maxlen = maxlen
    self.transform_ru = self.transform(self.vocab_ru)
    self.transform_en = self.transform(self.vocab_en)

  def transform(self, vocab):
    transforms = T.Sequential(
        T.VocabTransform(vocab),
        T.AddToken(token = vocab.get_stoi()['<SOS>'], begin = True),   # в начало
        T.AddToken(token = vocab.get_stoi()['<EOS>'], begin = False),  # в конец
        T.ToTensor(padding_value = vocab.get_stoi()['<PAD>']) ,  # заполнение 0
        T.PadTransform(max_length = self.maxlen, pad_value =  vocab.get_stoi()['<PAD>'])  # заполнение до фиксированной длины
    )
    return transforms


  def __getitem__(self, idx):

    if type(idx) == int:
      idx = slice(idx, idx+1)

    self.x = self.x[idx]
    self.y = self.y[idx]
    ru = self.transform_ru(self.x)
    en = self.transform_en(self.y)

    return ru, en



In [818]:
RuEnDataset(x = corpus_ru, y = corpus_en, vocab_ru = vocab_ru, vocab_en = vocab_en, maxlen = maxlen)[4]

(tensor([[   1,   20,    4,  225,   90, 1972, 3155,    2,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0]]),
 tensor([[   1,    6,  191,   13, 2117, 2033,   46,    2,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0]]))

In [819]:
# corpus_x = [
#         vocab_ru.lookup_indices(t)
#         for t in corpus_ru
#     ]
# # corpus_x

# transform = T.ToTensor(padding_value = 0)
# res1 = transform(corpus_x)
# res1[0]

In [820]:
train_ds = RuEnDataset(x = corpus_ru, y = corpus_en, vocab_ru = vocab_ru, vocab_en = vocab_en, maxlen = maxlen)[:]
test_ds = RuEnDataset(x = corpus_ru_test, y = corpus_en_test, vocab_ru = vocab_ru_test, vocab_en = vocab_en_test, maxlen = maxlen)[:]

In [821]:
train_ds[0].shape, test_ds[0].shape

(torch.Size([2330, 33]), torch.Size([580, 33]))

<p class="task" id="4"></p>

4\. Опишите модель `Encoder`, которая возвращает скрытое состояние рекуррентного слоя в соотстветствии со следующей схемой. Пропустите через эту модель первые 16 предложений на русском языке и выведите размер полученного результата на экран. Результатом должен являться тензор размера `1 x batch_size x hidden_dim` (если используется один однонаправленный рекуррентный слой и `batch_first=True`).

* количество эмбеддингов равно количеству слов на русском языке;
* размерность эмбеддингов выберите самостоятельно;
* при создании слоя эмбеддингов укажите `padding_idx`;
* размер скрытого состояния рекуррентного слоя выберите самостоятельно.

![encoder](https://i0.wp.com/www.adeveloperdiary.com/wp-content/uploads/2020/10/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-1.png?w=815&ssl=1)

- [ ] Проверено на семинаре

In [822]:
# Переносим данные на GPU, если доступен
device = th.device("cuda" if th.cuda.is_available() else "cpu")
device

device(type='cpu')

In [823]:
class Encoder(nn.Module):
  def __init__(self, embedding_dim, hidden_size):
      super().__init__()
      self.emb = nn.Embedding(
          num_embeddings = len(vocab_ru),
          embedding_dim=embedding_dim,
          padding_idx=0
      )
      # self.dropout = nn.Dropout(p=0.5)
      self.rnn = nn.GRU(embedding_dim, hidden_size, batch_first=True)

  def forward(self, X):
    out = self.emb(X) # batch x seq x emb_size
    # out = self.dropout(out)
    _, h = self.rnn(out) # out: batch x seq x hidden_size
    return h # 1 x batch x hidden_size

In [824]:
model = Encoder(embedding_dim = 150, hidden_size = 64)

In [825]:
train_ds[0][:16]

tensor([[   1,   30,  722, 3637, 5857,    2,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   1,    5,   28,  234, 3161, 3894, 1055,    2,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   1,    5,   61,  778, 1021,   16, 1983,    2,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   1,   14,   29,   65, 2805,    2,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   1,   20,    4,  225,   90, 1972, 3155,    2,    0,    0,    0,    0,
            0, 

In [826]:
encoder_output = model(train_ds[0][:16])
encoder_output.shape

torch.Size([1, 16, 64])

<p class="task" id="5"></p>

5\. Опишите модель `Decoder`, которая возвращает прогноз (набор индексов слов на английском языке). Пропустите через эту модель тензор скрытых состояний кодировщика, полученный в предыдущей задачи, и выведите размер полученного результата на экран. Результатом должен являться тензор размера `batch_size x seq_len x n_en_words` (если используется один однонаправленный рекуррентный слой и `batch_first=True`).

* количество эмбеддингов равно количеству слов на английском языке;
* размер выходного слоя равен количеству слов на английском языке;
* размерность эмбеддингов выберите самостоятельно;
* при создании слоя эмбеддингов укажите `padding_idx`;
* размер скрытого состояния рекуррентного слоя выберите самостоятельно.

![decoder](https://i2.wp.com/www.adeveloperdiary.com/wp-content/uploads/2020/10/Machine-Translation-using-Recurrent-Neural-Network-and-PyTorch-adeveloperdiary.com-2.png?w=899&ssl=1)

- [ ] Проверено на семинаре

In [827]:
class Decoder(nn.Module):
  def __init__(self, embedding_dim, decoder_hidden_size):
    super().__init__()
    self.emb = nn.Embedding(
          num_embeddings=len(vocab_en),
          embedding_dim=embedding_dim,
          padding_idx=0
    )
    self.rnn = nn.GRUCell(embedding_dim, decoder_hidden_size)
    self.fc = nn.Linear(decoder_hidden_size, len(vocab_en))

  def forward(self, encoder_output, labels):
    # labels: batch x seq_len - считаем, что в 0 столбце SOS
    # encoder_output: 1 x batch x encoder_hidden_size
    seq_len = labels.size(1)
    input_tokens = labels[:, 0]
    decoder_hidden = encoder_output[0]

    outputs = []
    for _ in range( seq_len):
      out = self.emb(input_tokens).relu() # batch x emb_size
      decoder_hidden = self.rnn(out, decoder_hidden) # batch x dec_hidden
      out = self.fc(decoder_hidden) # batch x n_en_tokens
      outputs.append(out)

      # teacher forcing
      # input_tokens = out.argmax(dim=1).detach()
      input_tokens = labels[:, _]  # входной токен для следующей итерации



    # вернуть прогнозы для каждого эл-та последовательности
    # batch x seq x n_en_token
    return th.stack(outputs, dim=1)

In [828]:
model = Decoder(embedding_dim = 150, decoder_hidden_size = 64)

In [829]:
train_ds[1][:16]

tensor([[   1,    6,  157,  248,   25, 4258,    2,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   1,    9,   52,    3,   64,  829,  503,   14, 1690,    2,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   1,    9,    5,    3,   49,    4,    3,   46, 1053,   23,  962,    2,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   1,    6,    5,    3,   19,    4,    3,   29,    4, 2583,    2,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0],
        [   1,    6,  191,   13, 2117, 2033,   46,    2,    0,    0,    0,    0,
            0, 

In [830]:
decoder_output = model(encoder_output, train_ds[1][:16])
decoder_output.shape

torch.Size([16, 33, 4348])

<p class="task" id="6"></p>

6\. Объедините модели `Encoder` и `Decoder` в одну модель `EncoderDecoder`. Пропустите через эту модель первые 16 предложений на русском языке и выведите размер полученного результата на экран. Сделайте полученный результат двумерным, объединив две первые размерности: `batch_size * seq_len x n_en_words`. Выведите размерность полученного результата на экран.

- [ ] Проверено на семинаре

In [831]:
class EncoderDecoder(nn.Module):

  def __init__(self, embedding_dim, hidden_size):
    super().__init__()
    self.encoder = Encoder(embedding_dim=embedding_dim, hidden_size=hidden_size)
    self.decoder = Decoder(embedding_dim=embedding_dim, decoder_hidden_size= hidden_size)

  def forward(self, input_encoder, input_decoder):
    encoder_out = self.encoder(input_encoder)
    decoder_out = self.decoder(encoder_out, input_decoder)

    return decoder_out.reshape(decoder_out.shape[0]*decoder_out.shape[1], decoder_out.shape[2])

In [832]:
model = EncoderDecoder(embedding_dim=150, hidden_size= 32)
out = model.forward(train_ds[0][:16], train_ds[1][:16]); out

tensor([[ 0.0754, -0.2473,  0.0626,  ..., -0.1175,  0.2293, -0.0419],
        [ 0.0749, -0.3136,  0.0356,  ..., -0.1943,  0.1759, -0.0399],
        [ 0.0962,  0.0873,  0.0377,  ..., -0.1410, -0.0304,  0.5648],
        ...,
        [ 0.0753, -0.0096,  0.2062,  ...,  0.0426,  0.2399, -0.0433],
        [ 0.0753, -0.0096,  0.2062,  ...,  0.0426,  0.2399, -0.0433],
        [ 0.0753, -0.0096,  0.2062,  ...,  0.0426,  0.2399, -0.0433]],
       grad_fn=<ViewBackward0>)

In [833]:
out.shape

torch.Size([528, 4348])

<p class="task" id="7"></p>

7\. Настройте модель, решив задачу классификации на основе прогнозов модели `EncoderDecoder`. Игнорируйте токен `<PAD>` при расчете ошибки. Во время обучения выводите на экран значения функции потерь для эпохи (на обучающем множестве), значение accuracy по токенам (на обучающем множестве) и пример перевода, сгенерированного моделью. После завершения обучения посчитайте BLEU для тестового множества.

- [ ] Проверено на семинаре

In [834]:
train_ds[0].shape,train_ds[1].shape

(torch.Size([2330, 33]), torch.Size([2330, 33]))

In [835]:
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(train_ds[0], train_ds[1])
test_dataset = TensorDataset(test_ds[0], test_ds[1])


loader_train = DataLoader(train_dataset, batch_size = 64, shuffle = True)
loader_test = DataLoader(test_dataset, batch_size = 64, shuffle = True)
next(iter(loader_train))
for ru_train, en_train in loader_train:
  print(ru_train.shape, en_train.shape)
  print(ru_train.view(-1))
  break

torch.Size([64, 33]) torch.Size([64, 33])
tensor([ 1,  5, 28,  ...,  0,  0,  0])


In [789]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

criterion = nn.CrossEntropyLoss(ignore_index = vocab_ru.get_stoi()['<PAD>'])
n_epochs = 100
model =  EncoderDecoder(embedding_dim=300, hidden_size= 32).to(device)
optimizer = optim.Adam(model.parameters(), lr = 0.01)

losses = []
for epoch in range(n_epochs):

  correct_train = 0  # правильно предсказанные
  total_train = 0

  for ru_train, en_train in loader_train:

    out = model(ru_train.to(device), en_train.to(device))
    # print(out)
    y_pred_train = th.argmax(out, dim=1)
    correct_train += (y_pred_train == en_train.view(-1)).sum().item()
    total_train += y_pred_train.shape[0]

    loss = criterion(out, en_train.view(-1))
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()


  losses.append(loss.item())
  if (epoch+1)% 10 == 0 or epoch == 0:
    print(f'Epoch №{epoch+1}, loss --> {loss.item()}')
    print(f'Accuracy -- > {correct_train/total_train}\n')

Epoch №1, loss --> 4.7959065437316895
Accuracy -- > 0.09553908180517623

Epoch №10, loss --> 3.0219926834106445
Accuracy -- > 0.15016256990505916

Epoch №20, loss --> 2.86721134185791
Accuracy -- > 0.16021589283391857

Epoch №30, loss --> 2.517676591873169
Accuracy -- > 0.16523605150214593

Epoch №40, loss --> 2.5480916500091553
Accuracy -- > 0.1680972818311874

Epoch №50, loss --> 2.14958119392395
Accuracy -- > 0.1718298868513461

Epoch №60, loss --> 2.237186908721924
Accuracy -- > 0.1769931070360255

Epoch №70, loss --> 2.0088024139404297
Accuracy -- > 0.18469241773962805

Epoch №80, loss --> 1.9338449239730835
Accuracy -- > 0.18835999479776303

Epoch №90, loss --> 2.0795071125030518
Accuracy -- > 0.1897645987774743

Epoch №100, loss --> 2.092564821243286
Accuracy -- > 0.1795942255169723



## Обратная связь
- [ ] Хочу получить обратную связь по решению